In [ ]:
"""
Only these lines need to be changed to the desired game
"""
folder = "output_young" # name of the folder, e.g. output_young
board_id = "1" # id of the board, e.g. "1" or "baseline"
description = "Team Blue: 2 guessers with baseline prompt<br/>Team Red: 2 guessers with \'young\' prompt" # any description showing up on the html

In [33]:
import json
import pandas as pd

In [34]:
class HistoryHTML:
    def __init__(self,game, output_path, description, header ="Codenames Game Replay"):
        # Initialize the HTML structure
        self.html = []
        self.initialize_html(header, description)

        self.create_html_file_from_history(game['history'], game['board'], game['winner'], output_path)


    def initialize_html(self, header, subheader):
        # Add basic structure for the HTML document
        self.html.append("""
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <title>Codenames Game Replay</title>
  <style>
    /* General Styles */
    body {
      font-family: Arial, sans-serif;
      line-height: 1.6;
      background-color: #f4f4f9;
      margin: 20px;
    }
    .header {
      text-align: center;
      margin-bottom: 20px;
    }
    .header h1 {
      color: #333;
    }
    .board {
      margin: 20px auto;
      display: grid;
      grid-template-columns: repeat(5, 1fr);
      gap: 10px;
      max-width: 600px;
      padding: 10px;
      border: 1px solid #ccc;
      border-radius: 10px;
      background-color: #fff;
    }
    .card {
      text-align: center;
      padding: 10px;
      border-radius: 5px;
      color: #fff;
      font-weight: bold;
    }
    .red { background-color: #ff6b6b; }
    .blue { background-color: #4dabf7; }
    .neutral { background-color: #adb5bd; color: #333; }
    .black { background-color: #333; }
    .log {
      margin: 20px auto;
      padding: 15px;
      background-color: #e9ecef;
      border-radius: 10px;
      max-width: 700px;
      box-shadow: 0 2px 5px rgba(0, 0, 0, 0.1);
    }
    .spymaster-blue, .guesser-blue, .spymaster-red, .guesser-red, .system {
      padding: 10px;
      border-radius: 5px;
      margin-bottom: 10px;
    }
    .spymaster-blue { background-color: #5f74de; }
    .guesser-blue { background-color: #b3d4fc; }
    .spymaster-red { background-color: #d14b59; }
    .guesser-red { background-color: #ffc9c9; }
    .system {
      text-align: center;
      font-weight: bold;
      color: #333;
      margin-top: 20px;
      background-color: #b0b0b0;
    }
  p {
    margin: 0;  /* Remove default margin around paragraphs */
    padding: 0; /* Optional: remove any padding, if necessary */
  }
  </style>
</head>
<body>"""+f"""
<div class="header">
  <h1>{header}</h1>
  <h2>{subheader}</h2>
</div>
<div class="board">
""")

    def add_board(self, board):
        # Add the game board to the HTML
        for word, color in board.items():
            self.html.append(f'<div class="card {color}">{word}</div>')
        self.html.append('</div>')

    def add_spymaster_clue(self, team, clue, count):
        # Add a spymaster clue
        self.html.append(f'<div class="spymaster-{team.lower()}"><strong>Spymaster - Clue:</strong> {clue}, {count}</div>')

    def add_discuss(self, player, team, content):
        # Add a discuss message
        self.html.append(f'<div class="guesser-{team.lower()}"><strong>{player} - Discuss:</strong><p>{content}</p></div>')

    def add_guess(self, player, team, guess, outcome):
        # Add a guess message
        self.html.append(f'<div class="guesser-{team.lower()}"><strong>{player} - Guess:</strong><p>{guess}</p>')
        if outcome:
            self.html.append(f'<p><strong>Outcome:</strong> {outcome}</p>')
        self.html.append('</div>')

    def add_end(self, player, team, reason):
        # Add an end message
        self.html.append(f'<div class="guesser-{team.lower()}"><strong>{player} - End:</strong><p>{reason}</p></div>')

    def add_system_message(self, message):
        # Add a system message
        self.html.append(f'<div class="system">{message}</div>')

    def finalize_html(self):
        # Add closing tags for the HTML document
        self.html.append("""
</body>
</html>
""")

    def get_html(self):
        # Return the full HTML as a string
        return ''.join(self.html)

    def create_html_file_from_history(self, history, board, winner, filename):
        # Create an HTML file from the history
        self.add_board(board)
        for round in history:
            self.add_spymaster_clue(round['team'], round['spymaster_clue'], round['number'])
            for action in round['actions']:
                if action['tool_call'] == 'codenames_discuss_tool':
                    self.add_discuss(action['teammember'], round['team'], action['tool_return'])
                elif action['tool_call'] == 'codenames_guess_tool':
                    self.add_guess(action['teammember'], round['team'], action['tool_args']['query'], action['outcome'])
                elif action['tool_call'] == 'codenames_end_turn_tool':
                    self.add_end(action['teammember'], round['team'], action['tool_return'])
                else:
                    raise Exception(f"Somenthing went wrong with this action:\n{action}")
        if winner:
            self.add_system_message(f'Team {winner} wins!')
        else:
            self.add_system_message('No winner! Game ended prematurely.')
        self.finalize_html()

        # save as html
        with open(filename, 'w') as f:
            f.write(self.get_html())  

        

In [36]:
with open(f"{folder}/all_results_stats.json", 'r') as f:
    all_results = json.load(f)
    result_df = pd.DataFrame(all_results)

game = result_df[result_df['board_id'] == str(board_id)].iloc[0]

history_html = HistoryHTML(game, f"{folder}/game_history_{board_id}.html", description)

## Create Dataframe of all all_results_stats combined
This functionality is not formally related to the rest of this file, but was useful to find interesting games to show

In [37]:
import pandas as pd
import os

# for each folder in the working directory, starting with 'output', read the all_results_stats.json file and append the results to a dataframe
df = None
for folder in os.listdir():
    #check if folder
    if not os.path.isdir(folder):
        continue
    if folder.startswith('output'):
        temp_df = pd.read_json(f"{folder}/all_results_stats.json")
        # add to datafram
        if df is None:
            df = temp_df
        else:
            df = pd.concat([df, temp_df])

df



,board,history,winner,config,board_id,guesses_red_list,guesses_blue_list,guesses_red_correct,guesses_red_incorrect_black,guesses_red_incorrect_neutral,guesses_red_incorrect_opponent,guesses_red_incorrect_invalid,guesses_blue_correct,guesses_blue_incorrect_black,guesses_blue_incorrect_neutral,guesses_blue_incorrect_opponent,guesses_blue_incorrect_invalid
0,"{'NOVEL': 'blue', 'SHAKESPEARE': 'black', 'LAS...","[{'team': 'RED', 'spymaster_clue': 'OPTICAL', ...",RED,riskavoiding,9,"[[OPTICAL, [['LASER', 'correct']]], [HIGH, [['...","[[FIBER, [['COTTON', 'correct'], ['NEEDLE', 'i...",4,0,1,1,0,5,1,0,3,0
1,"{'SHOT': 'blue', 'FILE': 'black', 'UNDERTAKER'...","[{'team': 'RED', 'spymaster_clue': 'Border', '...",RED,riskavoiding,5,"[[Border, [['CANADA', 'correct'], ['AZTEC', 'i...","[[MEDICAL, [['NURSE', 'correct'], ['AMBULANCE'...",5,0,1,1,0,6,1,1,3,0
2,"{'EUROPE': 'blue', 'HAM': 'blue', 'CANADA': 'r...","[{'team': 'RED', 'spymaster_clue': 'CARTOON', ...",RED,riskavoiding,4,"[[CARTOON, [['COMIC', 'correct']]], [NATION, [...","[[MEAT, [['HAM', 'correct'], ['DUCK', 'incorre...",7,0,1,1,0,6,0,3,2,0
3,"{'EGYPT': 'red', 'LAB': 'red', 'LAWYER': 'blue...","[{'team': 'RED', 'spymaster_clue': 'CITY', 'nu...",BLUE,riskavoiding,8,"[[CITY, [['LONDON', 'correct']]], [CLOTHES, [[...","[[BARRIER, [['FENCE', 'correct'], ['NET', 'cor...",6,0,0,1,0,7,0,3,0,0
4,"{'EYE': 'neutral', 'SATURN': 'blue', 'PORT': '...","[{'team': 'RED', 'spymaster_clue': 'EMERGENCY'...",RED,riskavoiding,3,"[[EMERGENCY, [['AMBULANCE', 'correct'], ['POLI...","[[MYTH, [['OCTOPUS', 'incorrect - neutral']]],...",7,0,1,1,0,3,1,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6,"{'EYE': 'neutral', 'SATURN': 'blue', 'PORT': '...","[{'team': 'RED', 'spymaster_clue': 'EMERGENCY'...",RED,gpt35-turbo_baseline,3,"[[EMERGENCY, [['AMBULANCE', 'correct'], ['AMBU...","[[MYTHICAL, [['OCTOPUS', 'incorrect - neutral'...",5,0,0,0,5,4,1,2,1,0
7,"{'LIGHT': 'blue', 'WASHINGTON': 'red', 'MERCUR...","[{'team': 'RED', 'spymaster_clue': 'FORTUNE', ...",RED,gpt35-turbo_baseline,0,"[[FORTUNE, [['LUCK', 'correct'], ['LUCK', 'inc...","[[MUSIC, [['OPERA', 'incorrect - opponent']]],...",8,0,1,0,4,7,0,1,1,1
8,"{'EGYPT': 'red', 'LAB': 'red', 'LAWYER': 'blue...","[{'team': 'RED', 'spymaster_clue': 'CITY', 'nu...",BLUE,gpt35-turbo_baseline,8,"[[CITY, [['LONDON', 'correct'], ['EGYPT', 'cor...","[[BARRIER, [['FENCE', 'correct'], ['NET', 'cor...",4,0,1,0,4,8,0,2,2,0
9,"{'NOVEL': 'blue', 'SHAKESPEARE': 'black', 'LAS...","[{'team': 'RED', 'spymaster_clue': 'OPTICAL', ...",RED,gpt35-turbo_baseline,9,"[[OPTICAL, [['LASER', 'correct'], ['TELESCOPE'...","[[FIBER, [['COTTON', 'correct'], ['NEEDLE', 'i...",3,0,1,0,4,5,1,1,3,0
